<a href="https://colab.research.google.com/github/JosephPLim/food_sales_predictions/blob/main/Food_Sales_AnalysisII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Food Sales Analysis II</center>

Joseph Lim


In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
#Basic Pandas/Numpy
import numpy as np
import pandas as pd

#Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

#ML Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
set_config(display='diagram')

#Regression Libraries
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#linking to Google Drive
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Dictionary

| Category                  | Data Type | Feature Type | Description                                                                                         |
|---------------------------|-----------|--------------|-----------------------------------------------------------------------------------------------------|
| Item_Identifier           | object    | Numeric      | Unique product ID                                                                                   |
| Item_Weight               | float     | Numeric      | Weight of product                                                                                   |
| Item_Fat_Content          | object    | Nominal      | Whether the product is low fat or regular                                                           |
| Item_Visibility           | float     | Numeric      | The percentage of total display area of all products in a store allocated to the particular product |
| Item_Type                 | object    | Nominal      | The category to which the product belongs                                                           |
| Item_MRP                  | float     | Numeric      | Maximum Retail Price (list price) of the product                                                    |
| Outlet_Identifier         | object    | Nominal      | Unique store ID                                                                                     |
| Outlet_Establishment_Year | int       | Ordinal      | The year in which store was established                                                             |
| Outlet_Size               | object    | Ordinal      | The size of the store in terms of ground area covered                                               |
| Outlet_Location_Type      | object    | Ordinal      | The type of area in which the store is located                                                      |
| Outlet_Type               | object    | Nominal      | Whether the outlet is a grocery store or some sort of supermarket                                   |
| Item_Outlet_Sales         | float     | TARGET       | Sales of the product in the particular store. This is the target variable to be predicted.          |

##Loading Data

In [2]:
filename = "/content/drive/MyDrive/eLearning/CodingDojo - Data Science/Portfolio Projects/sales_predictions.csv"
df = pd.read_csv(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [3]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##Data Cleaning
Dropping duplicates
Fixing inconsistencies in categorical columns

In [4]:
#number of rows and columns
df.shape

(8523, 12)

In [5]:
#Datatype for each variable
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [6]:
#drop duplicates
df.duplicated().sum()

0

In [7]:
for row in df:
  print(f"Column: {row}")
  print(df[row].value_counts())

Column: Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64
Column: Item_Weight
12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
7.275      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: Item_Weight, Length: 415, dtype: int64
Column: Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64
Column: Item_Visibility
0.000000    526
0.076975      3
0.162462      2
0.076841      2
0.073562      2
           ... 
0.013957      1
0.110460      1
0.124646      1
0.054142      1
0.044878      1
Name: Item_Visibility, Length: 7880, dtype: int64
Column: Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    

In [8]:
# Only categorical column that needs fixing is Item_Fat_Content
df["Item_Fat_Content"].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [9]:
#Fixing Item Fat Content to Low Fat and Regular Values
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("LF", "Low Fat")
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("low fat", "Low Fat")
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("reg", "Regular")
df["Item_Fat_Content"].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [10]:
#Obtaining the summary statistics of all numerical columns
df.describe().loc[["min", "max", "mean"]]

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
min,4.555000,0.000000,31.290000,1985.000000,33.290000
max,21.350000,0.328391,266.888400,2009.000000,13086.964800
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914


##Pre-Processing for ML

An aside regarding what might be considered ordinal columns (Outlet_Establishment_Year, Outlet_Size, Outlet_Location_Type): 

* I am choosing to utilize Outlet_Establishment_Year as a nominal column at the moment, changing the int datatypes to string datatypes. I feel this better reflects the years as categories rather than numbers for the model to utilize.

* After talking with Yvon Bilodeau, I was informed that Outlet_Location_Type and Outlet_Size did not seem to have a correlation with Item_Outlet_Sales. This will further be expanded upon in a "Food_Sales_Analysis" update at some point in the future to corrobrate this.

In [11]:
print(df["Outlet_Establishment_Year"].value_counts())
print(df["Outlet_Establishment_Year"].isna().sum())

1985    1463
1987     932
1999     930
1997     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: Outlet_Establishment_Year, dtype: int64
0


In [12]:
year_replacement_dict = {}
for i in range (1985, 2010):
  year_replacement_dict[i] = str(i)
year_replacement_dict
df["Outlet_Establishment_Year"].replace(year_replacement_dict, inplace = True)

In [13]:
print(df["Outlet_Size"].value_counts())
print(df["Outlet_Size"].isna().sum())

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64
2410


In [14]:
df["Outlet_Size"].fillna("Missing", inplace=True)
df["Outlet_Size"].value_counts()

Medium     2793
Missing    2410
Small      2388
High        932
Name: Outlet_Size, dtype: int64

###Feature and Target Identification

In [15]:
#Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant 
#variables as your features matrix.
y = df["Item_Outlet_Sales"]
X = df.drop(columns = "Item_Outlet_Sales")

In [16]:
#Perform a train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

###Preprocessing objects

In [22]:
#imputers
freq_imputer = SimpleImputer(strategy= "most_frequent")
mean_imputer = SimpleImputer(strategy= "mean")

#selectors
cat_selector = make_column_selector(dtype_include='object')
#cat_selector = ["Item_Identifier", "Item_Fat_Content", "Item_Type", "Outlet_Identifier", "Outlet_Establishment_Year",
#                "Outlet_Size", "Outlet_Location_Type", "Outlet_Type"]
num_selector = make_column_selector(dtype_include='number')
#num_selector = ["Item_Weight", "Item_Visibility", "Item_MRP"]

#scaler and one hot encoder
scaler = StandardScaler()
ohe = OneHotEncoder(sparse = "False", handle_unknown = "ignore")

In [18]:
numeric_pipe = make_pipeline(mean_imputer, scaler)
categorical_pipe = make_pipeline(freq_imputer, ohe)

In [23]:
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

preprocessor = make_column_transformer(number_tuple, category_tuple, remainder = "passthrough")

In [21]:
preprocessor.fit(X_train)

X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

##Regressions and Such

1. Your first task is to build a linear regression model to predict sales.
* Build a linear regression model.
Evaluate the performance of your model based on r^2.
Evaluate the performance of your model based on rmse.

2. Your second task is to build a regression tree model to predict sales.

* Build a simple regression tree model.
Compare the performance of your model based on r^2.
Compare the performance of your model based on rmse.
3. You now have tried 2 different models on your data set. You need to determine which model to implement.

* Overall, which model do you recommend?
Justify your recommendation.

###LINEAR REGRESSION MODEL

Evaluate the performance of your model based on r^2 and RMSE 

In [27]:
#Creating Regressor instance
reg = LinearRegression()

#Creating Pipeline from preprocessor to regressor instance
linreg_pipe = make_pipeline(preprocessor, reg)

# Fit the model pipeline on the training data
linreg_pipe.fit(X_train, y_train)
# Make predictions using the training and testing data
training_predictions = linreg_pipe.predict(X_train)
test_predictions = linreg_pipe.predict(X_test)

In [29]:
#Evaluating the performance of the model via r^2
train_r2 = r2_score(y_train, training_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"Model Training predictions r^2 = {train_r2}")
print(f"Model Testing predictions r^2 = {test_r2}")

#Evalating the performance of the model via RMSE
train_RMSE = np.sqrt(mean_squared_error(y_train, training_predictions))
test_RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

print(f"Model Training predictions RMSE = {train_RMSE}")
print(f"Model Testing predictions RMSE = {test_RMSE}")

Model Training predictions r^2 = 0.6716977904733816
Model Testing predictions r^2 = 0.38335523332222055
Model Training predictions RMSE = 985.6956156908806
Model Testing predictions RMSE = 1304.3423269636032


###REGRESSION TREE MODEL

Evaluate the performance of your model based on r^2 and RMSE

In [41]:
#model instance
dec_tree = DecisionTreeRegressor(random_state = 42)

#fitting the model
dec_tree.fit(X_train_processed, y_train)

#prediction values
dec_train_preds = dec_tree.predict(X_train_processed)
dec_test_preds = dec_tree.predict(X_test_processed)

#model evaluation
dec_train_score = dec_tree.score(X_train_processed, y_train)
dec_test_score = dec_tree.score(X_test_processed, y_test)
print(dec_train_score)
print(dec_test_score)

1.0
0.22903252442752497


In [42]:
#Decision Tree Tuning
depth = dec_tree.get_depth()
# List of values to try for max_depth:
depths = list(range(1, depth))
# Data frame to store the score for each value of max_depth:
scores = pd.DataFrame(index=depths, columns=['Test Score','Train Score'])
for depth in depths:
    dec_tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    dec_tree.fit(X_train_processed, y_train)
    train_score = dec_tree.score(X_train_processed, y_train)
    test_score = dec_tree.score(X_test_processed, y_test)
    scores.loc[depth, 'Train Score'] = train_score
    scores.loc[depth, 'Test Score'] = test_score

sorted_scores = scores.sort_values(by='Test Score', ascending=False)
sorted_scores.head()

,Test Score,Train Score
5,0.596056,0.604207
6,0.584668,0.615702
4,0.583937,0.582705
7,0.577712,0.629269
8,0.559441,0.641334


In [47]:
#Decision Tree Tuning 2; optimal depth = 5
optimal_depth = 5

dec_tree_opt = DecisionTreeRegressor(max_depth = optimal_depth, random_state = 42)
dec_tree_opt.fit(X_train_processed, y_train)

#Evaluating the performance of the model via r^2
dec_tree_r2_train = r2_score(y_train, dec_tree_opt.predict(X_train_processed))
dec_tree_r2_test = r2_score(y_test, dec_tree_opt.predict(X_test_processed))

print(f"Model Training predictions r^2 = {train_r2}")
print(f"Model Testing predictions r^2 = {test_r2}")

#Evalating the performance of the model via RMSE
train_RMSE = np.sqrt(mean_squared_error(y_train, dec_tree_opt.predict(X_train_processed)))
test_RMSE = np.sqrt(mean_squared_error(y_test, dec_tree_opt.predict(X_test_processed)))

print(f"Model Training predictions RMSE = {train_RMSE}")
print(f"Model Testing predictions RMSE = {test_RMSE}")

Model Training predictions r^2 = 0.6716977904733816
Model Testing predictions r^2 = 0.38335523332222055
Model Training predictions RMSE = 1082.281287111312
Model Testing predictions RMSE = 1055.6851402178304


####Which model to use?

You now have tried 2 different models on your data set. You need to determine which model to implement.
Overall, which model do you recommend? Justify your recommendation.


Linear Regression:

Model Training predictions r^2 = 0.6716977904733816

Model Testing predictions r^2 = 0.38335523332222055

Model Training predictions RMSE = 985.6956156908806

Model Testing predictions RMSE = 1304.3423269636032

Decision Tree:

Model Training predictions r^2 = 0.6716977904733816

Model Testing predictions r^2 = 0.38335523332222055

Model Training predictions RMSE = 1082.281287111312

Model Testing predictions RMSE = 1055.6851402178304

Whe r^2 values for both the linear regression and the decision tree are the same. Howver the RMSEs for both are different. Because the RMSE for the test on the decision tree is lower than the RMSE for the test on the regression, I recommend the Decision Tree in order to avoid larger errors.